In [52]:
!pip install -r requirements.txt

In [53]:
!pip install  opencv-python mediapipe matplotlib

In [2]:
import os
import matplotlib.pyplot as plt

from PIL import Image
import cv2

import mediapipe as mp
from mediapipe.python.solutions import pose as mp_pose

#if you are using colab
# from google.colab.patches import cv2_imshow

# PyTorch Hub
import torch

In [3]:
from ultralytics import YOLO

# Model
yolo_model = YOLO('yolov8n.pt')

#since we are only intrested in detecting person
yolo_model.classes=[0]

In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [5]:
# Prompt user for the video name
video_name = input("Введите название видео (без расширения .mp4): ")

video_path = f'./data/videos/input/{video_name}.mp4'
output_path = f'./data/videos/output/{video_name}_output.mp4'

In [6]:
import cv2

#get the dimension of the video
cap = cv2.VideoCapture('video_path')
if cap.isOpened():
    w = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    h = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    size = (w, h)
    print(size)

In [7]:
cap = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

In [8]:
import os


cap = cv2.VideoCapture(video_path)

#for webacam cv2.VideoCapture(NUM) NUM -> 0,1,2 for primary and secondary webcams..

#For saving the video file as output.avi
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

max_file_size_mb = 20  # Максимальный размер файла в мегабайтах
max_file_size_bytes = max_file_size_mb * 1024 * 1024  # Перевод в байты

current_file_size = os.path.getsize(output_path)
while cap.isOpened() and current_file_size < max_file_size_bytes:
    ret, frame = cap.read()
    if not ret:
      break
    # Recolor Feed from RGB to BGR
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    #making image writeable to false improves prediction
    image.flags.writeable = False    
    
    result = yolo_model(image)    
    
    # Recolor image back to BGR for rendering
    image.flags.writeable = True   
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    # print(result.xyxy)  # img1 predictions (tensor)

    #This array will contain crops of images incase we need it 
    img_list =[]
    
    #we need some extra margin bounding box for human crops to be properly detected
    MARGIN=10

    # Print the result object to inspect its structure
    print(result)
    
    # Access the bounding boxes based on the actual structure of the result object
    for bbox in result[0].boxes:
        print(bbox.xyxy)  # Adjust this line based on the actual keys in the result object
        xmin, ymin, xmax, ymax = bbox.xyxy.cpu().numpy()[0].astype(int)
        xmin -= 10
        ymin -= 10
        xmax += 10
        ymax += 10
        # xmin, ymin, xmax, ymax = detection['bbox']  # Adjust this line based on the actual keys in the result object
        confidence = bbox.conf
        clas = bbox.cls

        with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
            # Media pose prediction
            results = pose.process(image[int(ymin)+MARGIN:int(ymax)+MARGIN, int(xmin)+MARGIN:int(xmax)+MARGIN])


        #Draw landmarks on image, if this thing is confusing please consider going through numpy array slicing 
        mp_drawing.draw_landmarks(image[int(ymin)+MARGIN:int(ymax)+MARGIN,int(xmin)+MARGIN:int(xmax)+MARGIN:], results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                              ) 
        img_list.append(image[int(ymin):int(ymax),int(xmin):int(xmax):])
    # cv2_imshow(image)

    # writing in the video file 
    out.write(image)

    ## Code to quit the video incase you are using the webcam             
    # cv2.imshow('Activity recognition', image)
    # if cv2.waitKey(10) & 0xFF == ord('q'):
    #     break

cap.release()
out.release()
cv2.destroyAllWindows()

# После записи кадра обновляем текущий размер файла
current_file_size = os.path.getsize(output_path)



0: 384x640 18 persons, 1 handbag, 61.7ms
Speed: 3.1ms preprocess, 61.7ms inference, 140.1ms postprocess per image at shape (1, 3, 384, 640)
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 

D:\Users\Legion\BIA Technologies\YOLO_8_test\.venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


tensor([[1542.7061,  615.3635, 1613.6946,  799.6339]], device='cuda:0')
tensor([[1789.8135,  722.6413, 1872.7053,  912.8868]], device='cuda:0')
tensor([[899.3494, 737.5556, 982.9407, 831.0342]], device='cuda:0')
tensor([[1147.5393,  617.9464, 1217.1809,  793.6445]], device='cuda:0')
tensor([[1258.8828,  623.7272, 1324.6737,  820.5582]], device='cuda:0')
tensor([[503.3204, 285.4629, 598.8235, 462.4557]], device='cuda:0')
tensor([[518.4659, 411.5782, 631.5681, 653.0865]], device='cuda:0')
tensor([[ 252.7481,  701.9124,  350.7668, 1020.4261]], device='cuda:0')
tensor([[1644.1768,  801.7879, 1684.6558,  848.4662]], device='cuda:0')
tensor([[ 585.1331,  982.6967,  723.8555, 1080.0000]], device='cuda:0')
tensor([[ 992.3959,  750.2510, 1052.1653,  844.2233]], device='cuda:0')
tensor([[144.0905, 142.7240, 269.4500, 418.1348]], device='cuda:0')
tensor([[1057.4606,  616.5933, 1103.6494,  731.1615]], device='cuda:0')
tensor([[133.4121, 346.7308, 272.8166, 684.8690]], device='cuda:0')
tensor([[472

KeyboardInterrupt: 

In [101]:
import os


cap = cv2.VideoCapture(video_path)

#for webcam cv2.VideoCapture(NUM) NUM -> 0,1,2 for primary and secondary webcams..

#For saving the video file as output.avi
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

max_file_size_mb = 20  # Максимальный размер файла в мегабайтах
max_file_size_bytes = max_file_size_mb * 1024 * 1024  # Перевод в байты

current_file_size = os.path.getsize(output_path)
while cap.isOpened() and current_file_size < max_file_size_bytes:
    ret, frame = cap.read()
    if not ret:
      break
    # Recolor Feed from RGB to BGR
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    #making image writeable to false improves prediction
    image.flags.writeable = False    
    
    result = yolo_model.predict(frame)
    for res in result:
        for box in res.boxes:
            print(box.xyxy, box.cls, box.conf)
    # result = yolo_model(image)
cap.release()
out.release()
cv2.destroyAllWindows()
    
#     # Recolor image back to BGR for rendering
#     image.flags.writeable = True   
#     image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
#     # print(result.xyxy)  # img1 predictions (tensor)
# 
#     #This array will contain crops of images incase we need it 
#     img_list =[]
#     
#     #we need some extra margin bounding box for human crops to be properly detected
#     MARGIN=10
# 
#     # Print the result object to inspect its structure
#     print(result)
#     
#     # Access the bounding boxes based on the actual structure of the result object
#     for bbox in result[0].boxes:
#         print(bbox.xyxy)  # Adjust this line based on the actual keys in the result object
#         xmin, ymin, xmax, ymax = bbox.xyxy.cpu().numpy()[0].astype(int)
#         # xmin, ymin, xmax, ymax = detection['bbox']  # Adjust this line based on the actual keys in the result object
#         confidence = bbox.conf
#         clas = bbox.cls
# 
#         with mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.3) as pose:
#             # Media pose prediction
#             results = pose.process(image[int(ymin)+MARGIN:int(ymax)+MARGIN, int(xmin)+MARGIN:int(xmax)+MARGIN])
# 
# 
#         #Draw landmarks on image, if this thing is confusing please consider going through numpy array slicing 
#         mp_drawing.draw_landmarks(image[int(ymin)+MARGIN:int(ymax)+MARGIN,int(xmin)+MARGIN:int(xmax)+MARGIN:], results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
#                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
#                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
#                               ) 
#         img_list.append(image[int(ymin):int(ymax),int(xmin):int(xmax):])
#     # cv2_imshow(image)
# 
#     # writing in the video file 
#     out.write(image)
# 
#     ## Code to quit the video incase you are using the webcam             
#     # cv2.imshow('Activity recognition', image)
#     # if cv2.waitKey(10) & 0xFF == ord('q'):
#     #     break
# 
# cap.release()
# out.release()
# cv2.destroyAllWindows()
# 
# # После записи кадра обновляем текущий размер файла
# current_file_size = os.path.getsize(output_path)


In [ ]:
model = YOLO("models/yolov8m.pt")
cap = cv2.VideoCapture("jump.mp4")
while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        results = model.predict(frame)
        for res in results:
            for box in res.boxes:
                print(box.xyxy, box.cls, box.conf)
    else:
        break

cap.release()

In [77]:
import os


cap = cv2.VideoCapture(video_path)

#for webacam cv2.VideoCapture(NUM) NUM -> 0,1,2 for primary and secondary webcams..

#For saving the video file as output.avi
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

max_file_size_mb = 20  # Максимальный размер файла в мегабайтах
max_file_size_bytes = max_file_size_mb * 1024 * 1024  # Перевод в байты

current_file_size = os.path.getsize(output_path)
while cap.isOpened() and current_file_size < max_file_size_bytes:
    ret, frame = cap.read()
    if not ret:
      break
    # Recolor Feed from RGB to BGR
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    #making image writeable to false improves prediction
    image.flags.writeable = False    
    
    result = yolo_model.predict(frame)    
    
    # Recolor image back to BGR for rendering
    image.flags.writeable = True   
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    # print(result.xyxy)  # img1 predictions (tensor)

    #This array will contain crops of images incase we need it 
    img_list =[]
    
    #we need some extra margin bounding box for human crops to be properly detected
    MARGIN=10

    # Print the result object to inspect its structure
    print(result)
    
    # Access the bounding boxes based on the actual structure of the result object
    for res in result:
        for bbox in res.boxes:
            print(bbox.xyxy, bbox.cls, bbox.conf)  # Adjust this line based on the actual keys in the result bject
            xmin, ymin, xmax, ymax = bbox.xyxy.cpu().numpy()[0].astype(int)
            # xmin, ymin, xmax, ymax = detection['bbox']  # Adjust this line based on the actual keys in the result object
            confidence = bbox.conf
            clas = bbox.cls

            with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
                # Media pose prediction
                results = pose.process(image[int(ymin)+MARGIN:int(ymax)+MARGIN, int(xmin)+MARGIN:int(xmax)+MARGIN])


            #Draw landmarks on image, if this thing is confusing please consider going through numpy array slicing 
            mp_drawing.draw_landmarks(image[int(ymin)+MARGIN:int(ymax)+MARGIN,int(xmin)+MARGIN:int(xmax)+MARGIN:], results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                  ) 
            img_list.append(image[int(ymin):int(ymax),int(xmin):int(xmax):])
        # cv2_imshow(image)

        # writing in the video file 
        out.write(image)

        ## Code to quit the video incase you are using the webcam             
        # cv2.imshow('Activity recognition', image)
        # if cv2.waitKey(10) & 0xFF == ord('q'):
        #     break

cap.release()
out.release()
cv2.destroyAllWindows()

# После записи кадра обновляем текущий размер файла
current_file_size = os.path.getsize(output_path)



0: 384x640 19 persons, 1 airplane, 1 handbag, 1 chair, 8.9ms
Speed: 3.0ms preprocess, 8.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple',